In [1]:
import json
import pandas as pd
import gensim
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import numpy as np
import math

# Creating Corpus

In [2]:
market_data = pd.read_csv('data/market_data.csv')
with open('json/readable.json') as json_file:
    heads = json.load(json_file)

In [6]:
head_dates =list(heads.keys()) #gets the dates into a list
dates = [d for d in market_data['Date'] if d.replace("-","") in heads.keys()]

vix_close = [market_data.loc[i].values[4] for i in range(len(market_data)) if market_data['Date'][i] in dates]
dji_close = [market_data.loc[i].values[-3] for i in range(len(market_data)) if market_data['Date'][i] in dates]

corpus = [''.join([" . ".join(heads[dates[i+k].replace("-","")]) for k in range(5)])
           for i in range(len(dates)-5)]

# Create tf-idf Scores for '5 Day Advance Headlines'

In [7]:
vectorizer = TfidfVectorizer(min_df=0.01, max_df=0.2,  norm=None) #get vector and tdidf scores
tfidf_scores = vectorizer.fit_transform(corpus)

In [17]:
tfidf_matrix = tfidf_scores.toarray() #put it to a matrix
df_tfidf_scores = pd.DataFrame(tfidf_matrix, columns=vectorizer.get_feature_names())
df_tfidf_scores.head()

,01,02,03,04,05,06,07,08,09,1000,...,zones,zoning,zoo,zoom,zte,zuckerberg,zuckerman,zuma,zurich,zynga
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# VXX

In [24]:
y = vix_close[5::]
X_train, X_test, y_train, y_test = train_test_split(tfidf_matrix, y, test_size=0.3, random_state=0)

clf = KernelRidge(alpha=1,kernel = "poly")
clf.fit(X_train, y_train)
pred =clf.predict(X_test)
print("R^2 Score: " + str(clf.score(X_test,y_test)))
print("R : " + str(math.sqrt(clf.score(X_test,y_test))))
print("MSE: " + str(mean_squared_error(pred,y_test)))

R^2 Score: 0.7895515205591691
R : 0.888567116519157
MSE: 15.319763108194207


In [52]:
y = dji_close[5::]
X_train, X_test, y_train, y_test = train_test_split(tfidf_matrix, y, test_size=0.3, random_state=0)

clf = KernelRidge(alpha=1,kernel = "poly")
clf.fit(X_train, y_train)
pred =clf.predict(X_test)
print("R^2 Score: " + str(clf.score(X_test,y_test)))
print("R : " + str(math.sqrt(clf.score(X_test,y_test))))
print("MSE: " + str(mean_squared_error(pred,y_test)))

R^2 Score: 0.9098779712104513
R : 0.9538752388077024
MSE: 2562283.0350912996
